<a href="https://colab.research.google.com/github/reniamorf/CCS2project_MultimodalADdetection_PUBLIC/blob/main/features_extraction_STEP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# display options for wide tables
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

In [ ]:
BASE_DIR = Path("/content/drive/MyDrive/CAMES/pilot_testing/pilot_data/julie_1712_eyetracking_gsr_screenrec")

GAZE_FILE = BASE_DIR / "gaze_with_pupil.tsv"
FIX_FILE  = BASE_DIR / "fixations.csv"
SAC_FILE  = BASE_DIR / "saccades.csv"
UI_FILE   = BASE_DIR / "ui_events_unix.csv"
PERF_FILE = BASE_DIR / "Julie1712_performance_metrics.csv"

gaze = pd.read_csv(GAZE_FILE, sep="\t")
fix  = pd.read_csv(FIX_FILE)
sac  = pd.read_csv(SAC_FILE)
ui   = pd.read_csv(UI_FILE)
perf = pd.read_csv(PERF_FILE)

print("Loaded shapes:")
print("gaze:", gaze.shape)
print("fix:", fix.shape)
print("sac:", sac.shape)
print("ui:", ui.shape)
print("perf:", perf.shape)


Loaded shapes:
gaze: (293970, 36)
fix: (6610, 14)
sac: (6609, 14)
ui: (86, 5)
perf: (7, 6)


In [ ]:
# --- sanity checks ---
assert "time_s" in gaze.columns, "gaze must contain time_s"
assert "task_id" in gaze.columns, "gaze must contain task_id"
assert "event_time_s" in ui.columns, "ui must contain event_time_s"
assert "start" in fix.columns and "end" in fix.columns, "fix must contain start/end"
assert "start" in sac.columns and "end" in sac.columns, "sac must contain start/end"

dt = gaze["time_s"].diff().median()
freq = 1.0 / dt
print("Median dt (s):", float(dt))
print("Estimated freq (Hz):", float(freq))

print("Gaze time range (s):", float(gaze["time_s"].min()), "→", float(gaze["time_s"].max()))
print("Tasks in gaze:", sorted(gaze["task_id"].dropna().unique()))
print("UI event types:\n", ui["event_type"].value_counts())


Median dt (s): 0.011103000000048269
Estimated freq (Hz): 90.06574799564557
Gaze time range (s): 0.0 → 3468.451745
Tasks in gaze: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0)]
UI event types:
 event_type
click_button    43
click_next      34
click_photo      9
Name: count, dtype: int64


In [ ]:
# --- convert indices to time_s ---
n_gaze = len(gaze)

for df, name in [(fix, "fix"), (sac, "sac")]:
    df["start"] = df["start"].astype(int)
    df["end"]   = df["end"].astype(int)
    assert df["start"].max() < n_gaze and df["end"].max() < n_gaze, f"{name} indices out of bounds"

# Map to session time
fix["start_time_s"] = gaze.loc[fix["start"], "time_s"].values
fix["end_time_s"]   = gaze.loc[fix["end"],   "time_s"].values
sac["start_time_s"] = gaze.loc[sac["start"], "time_s"].values
sac["end_time_s"]   = gaze.loc[sac["end"],   "time_s"].values

# Durations in seconds (ET outputs dur in ms)
if "dur" in fix.columns:
    fix["dur_s"] = fix["dur"] / 1000.0
if "dur" in sac.columns:
    sac["dur_s"] = sac["dur"] / 1000.0

# Task assignment via gaze sample at 'start'
fix["task_id"] = gaze.loc[fix["start"], "task_id"].values
sac["task_id"] = gaze.loc[sac["start"], "task_id"].values

print("Fix time range (s):", float(fix["start_time_s"].min()), "→", float(fix["end_time_s"].max()))
print("Sac time range (s):", float(sac["start_time_s"].min()), "→", float(sac["end_time_s"].max()))
print("Fix missing task_id:", int(fix["task_id"].isna().sum()))
print("Sac missing task_id:", int(sac["task_id"].isna().sum()))


Fix time range (s): 0.0 → 3468.440642
Sac time range (s): 0.088825 → 3468.32961
Fix missing task_id: 1272
Sac missing task_id: 1273


In [ ]:
perf["task_id"] = (
    perf["Task_ID"]
    .astype(str)
    .str.extract(r"(\d+)")
    .astype(float)
    .astype("Int64")
)

print(perf[["Task_ID","task_id","Correct","Total","Accuracy"]])


  Task_ID  task_id  Correct  Total  Accuracy
0  task_1        1        1      2      0.50
1  task_2        2        1      1      1.00
2  task_3        3        1      3      0.33
3  task_4        4        0      1      0.00
4  task_5        5        2      7      0.29
5  task_6        6        2      7      0.29
6  task_7        7        1      3      0.33


In [ ]:
# ===============================
# TONIC TASK FEATURES
# ===============================

PUPIL_COL = "pupil_smooth" if "pupil_smooth" in gaze.columns else "pupil_interp"

# Ensure blink columns are boolean (TSV reload can change dtype)
gaze["blink"] = gaze["blink"].astype(bool)
gaze["blink_onset"] = gaze["blink_onset"].astype(bool)

task_rows = []

for tid in sorted(gaze["task_id"].dropna().unique()):
    g = gaze[gaze["task_id"] == tid].copy()
    if g.empty:
        continue

    t_start = g["time_s"].min()
    t_end   = g["time_s"].max()
    dur_s   = t_end - t_start

    # --- pupil baseline (first 2 s) ---
    base_mask = (g["time_s"] >= t_start) & (g["time_s"] < t_start + 2)
    baseline = g.loc[base_mask, PUPIL_COL].mean()
    pupil_mean = (g[PUPIL_COL] - baseline).mean()

    # --- pupil slope (fit only on valid samples; polyfit can't handle NaNs) ---
    m = g["time_s"].notna() & g[PUPIL_COL].notna()
    if m.sum() > 10:
        pupil_slope = np.polyfit(
            g.loc[m, "time_s"].to_numpy(),
            g.loc[m, PUPIL_COL].to_numpy(),
            1
        )[0]
    else:
        pupil_slope = np.nan

    # --- blink metrics ---
    # time-loss metric: fraction of samples during blinks
    blink_time_frac = g["blink"].mean()
    # physiological blink rate: blink onsets per minute
    blink_per_min = g["blink_onset"].sum() / (dur_s / 60.0) if dur_s > 0 else np.nan

    # --- fixation metrics ---
    f = fix[fix["task_id"] == tid]
    fix_rate_hz = len(f) / dur_s if dur_s > 0 else np.nan
    fix_dur_mean_s = f["dur_s"].mean() if not f.empty else np.nan

    # --- saccade metrics (true S only) ---
    s = sac[(sac["task_id"] == tid) & (sac["sclass"] == "S")]
    sac_rate_hz = len(s) / dur_s if dur_s > 0 else np.nan
    sac_amp_mean_px = s["amp"].mean() if not s.empty else np.nan

    task_rows.append({
        "task_id": int(tid),
        "task_duration_s": dur_s,
        "pupil_mean_bc": pupil_mean,
        "pupil_slope": pupil_slope,
        "blink_time_frac": blink_time_frac,
        "blink_per_min": blink_per_min,
        "fix_rate_hz": fix_rate_hz,
        "fix_dur_mean_s": fix_dur_mean_s,
        "sac_rate_hz": sac_rate_hz,
        "sac_amp_mean_px": sac_amp_mean_px
    })

task_features = pd.DataFrame(task_rows).sort_values("task_id").reset_index(drop=True)
display(task_features)


,task_id,task_duration_s,pupil_mean_bc,pupil_slope,blink_time_frac,blink_per_min,fix_rate_hz,fix_dur_mean_s,sac_rate_hz,sac_amp_mean_px
0,1,358.990243,0.522891,0.000691,0.086539,14.039379,1.791135,0.471891,1.454079,219.622147
1,2,209.060412,1.165456,0.000205,0.059017,13.201926,1.894189,0.458705,1.511525,197.351515
2,3,389.439403,0.588936,-0.000222,0.067974,12.941680,2.154379,0.394670,1.720422,199.446432
3,4,464.259411,0.620132,-0.000132,0.116114,9.563619,1.432389,0.574591,1.038213,160.520821
4,5,456.654522,0.863760,0.001050,0.023232,6.438127,1.151855,0.443323,0.915353,165.621567
5,6,407.495757,0.695545,-0.000255,0.033748,13.104431,2.508002,0.345782,2.007383,230.752001
6,7,381.418928,0.570387,-0.000119,0.108719,15.888042,2.231143,0.357596,1.782817,253.831161
7,8,264.633045,0.385534,-0.000282,0.018133,7.935517,1.496412,0.620149,1.239452,176.636879


In [ ]:
#so above some corrections need to be made bc there is no merge with performance scores and no correction for task 5 which should be deleted and substituted by task 6 etc bc there was a technical issue so task 5 was done twice, only the second time fully!!!!!!

In [ ]:
# ===============================
# TONIC TASK FEATURES
# ===============================

PUPIL_COL = "pupil_smooth" if "pupil_smooth" in gaze.columns else "pupil_interp"

# Ensure blink columns are boolean (TSV reload can change dtype)
gaze["blink"] = gaze["blink"].astype(bool)
gaze["blink_onset"] = gaze["blink_onset"].astype(bool)

task_rows = []

for tid in sorted(gaze["task_id"].dropna().unique()):
    g = gaze[gaze["task_id"] == tid].copy()
    if g.empty:
        continue

    t_start = g["time_s"].min()
    t_end   = g["time_s"].max()
    dur_s   = t_end - t_start

    # --- pupil baseline (first 2 s) ---
    base_mask = (g["time_s"] >= t_start) & (g["time_s"] < t_start + 2)
    baseline = g.loc[base_mask, PUPIL_COL].mean()
    pupil_mean = (g[PUPIL_COL] - baseline).mean()

    # --- pupil slope (fit only on valid samples; polyfit can't handle NaNs) ---
    m = g["time_s"].notna() & g[PUPIL_COL].notna()
    if m.sum() > 10:
        pupil_slope = np.polyfit(
            g.loc[m, "time_s"].to_numpy(),
            g.loc[m, PUPIL_COL].to_numpy(),
            1
        )[0]
    else:
        pupil_slope = np.nan

    # --- blink metrics ---
    # time-loss metric: fraction of samples during blinks
    blink_time_frac = g["blink"].mean()
    # physiological blink rate: blink onsets per minute
    blink_per_min = g["blink_onset"].sum() / (dur_s / 60.0) if dur_s > 0 else np.nan

    # --- fixation metrics ---
    f = fix[fix["task_id"] == tid]
    fix_rate_hz = len(f) / dur_s if dur_s > 0 else np.nan
    fix_dur_mean_s = f["dur_s"].mean() if not f.empty else np.nan

    # --- saccade metrics (true S only) ---
    s = sac[(sac["task_id"] == tid) & (sac["sclass"] == "S")]
    sac_rate_hz = len(s) / dur_s if dur_s > 0 else np.nan
    sac_amp_mean_px = s["amp"].mean() if not s.empty else np.nan

    task_rows.append({
        "task_id": int(tid),
        "task_duration_s": dur_s,
        "pupil_mean_bc": pupil_mean,
        "pupil_slope": pupil_slope,
        "blink_time_frac": blink_time_frac,
        "blink_per_min": blink_per_min,
        "fix_rate_hz": fix_rate_hz,
        "fix_dur_mean_s": fix_dur_mean_s,
        "sac_rate_hz": sac_rate_hz,
        "sac_amp_mean_px": sac_amp_mean_px
    })

task_features = pd.DataFrame(task_rows).sort_values("task_id").reset_index(drop=True)
display(task_features)


,task_id,task_duration_s,pupil_mean_bc,pupil_slope,blink_time_frac,blink_per_min,fix_rate_hz,fix_dur_mean_s,sac_rate_hz,sac_amp_mean_px
0,1,358.990243,0.522891,0.000691,0.086539,14.039379,1.791135,0.471891,1.454079,219.622147
1,2,209.060412,1.165456,0.000205,0.059017,13.201926,1.894189,0.458705,1.511525,197.351515
2,3,389.439403,0.588936,-0.000222,0.067974,12.941680,2.154379,0.394670,1.720422,199.446432
3,4,464.259411,0.620132,-0.000132,0.116114,9.563619,1.432389,0.574591,1.038213,160.520821
4,5,456.654522,0.863760,0.001050,0.023232,6.438127,1.151855,0.443323,0.915353,165.621567
5,6,407.495757,0.695545,-0.000255,0.033748,13.104431,2.508002,0.345782,2.007383,230.752001
6,7,381.418928,0.570387,-0.000119,0.108719,15.888042,2.231143,0.357596,1.782817,253.831161
7,8,264.633045,0.385534,-0.000282,0.018133,7.935517,1.496412,0.620149,1.239452,176.636879


In [ ]:
task_features["participant_id"] = "P01"


In [ ]:
display(task_features)

,task_id,task_duration_s,pupil_mean_bc,pupil_slope,blink_time_frac,blink_per_min,fix_rate_hz,fix_dur_mean_s,sac_rate_hz,sac_amp_mean_px,participant_id
0,1,358.990243,0.522891,0.000691,0.086539,14.039379,1.791135,0.471891,1.454079,219.622147,P01
1,2,209.060412,1.165456,0.000205,0.059017,13.201926,1.894189,0.458705,1.511525,197.351515,P01
2,3,389.439403,0.588936,-0.000222,0.067974,12.941680,2.154379,0.394670,1.720422,199.446432,P01
3,4,464.259411,0.620132,-0.000132,0.116114,9.563619,1.432389,0.574591,1.038213,160.520821,P01
4,5,456.654522,0.863760,0.001050,0.023232,6.438127,1.151855,0.443323,0.915353,165.621567,P01
5,6,407.495757,0.695545,-0.000255,0.033748,13.104431,2.508002,0.345782,2.007383,230.752001,P01
6,7,381.418928,0.570387,-0.000119,0.108719,15.888042,2.231143,0.357596,1.782817,253.831161,P01
7,8,264.633045,0.385534,-0.000282,0.018133,7.935517,1.496412,0.620149,1.239452,176.636879,P01


TONIC FEATURES COMPUTION

In [ ]:
gaze["blink"] = gaze["blink"].astype(bool)
gaze["blink_onset"] = gaze["blink_onset"].astype(bool)

In [ ]:
PUPIL_COL = "pupil_smooth" if "pupil_smooth" in gaze.columns else "pupil_interp"

task_rows = []

for tid in sorted(gaze["task_id"].dropna().unique()):
    g = gaze[gaze["task_id"] == tid].copy()
    if g.empty:
        continue

    t_start = g["time_s"].min()
    t_end   = g["time_s"].max()
    dur_s   = t_end - t_start

    # --- pupil baseline (first 2 s) ---
    base_mask = (g["time_s"] >= t_start) & (g["time_s"] < t_start + 2)
    baseline = g.loc[base_mask, PUPIL_COL].mean()
    pupil_mean = (g[PUPIL_COL] - baseline).mean()

    # --- pupil slope (valid samples only) ---
    m = g["time_s"].notna() & g[PUPIL_COL].notna()
    if m.sum() > 10:
        pupil_slope = np.polyfit(
            g.loc[m, "time_s"].to_numpy(),
            g.loc[m, PUPIL_COL].to_numpy(),
            1
        )[0]
    else:
        pupil_slope = np.nan

    # --- blink metrics ---
    blink_time_frac = g["blink"].mean()
    blink_per_min = g["blink_onset"].sum() / (dur_s / 60.0) if dur_s > 0 else np.nan

    # --- fixation metrics ---
    f = fix[fix["task_id"] == tid]
    fix_rate_hz = len(f) / dur_s if dur_s > 0 else np.nan
    fix_dur_mean_s = f["dur_s"].mean() if not f.empty else np.nan

    # --- saccade metrics (true S only) ---
    s = sac[(sac["task_id"] == tid) & (sac["sclass"] == "S")]
    sac_rate_hz = len(s) / dur_s if dur_s > 0 else np.nan
    sac_amp_mean_px = s["amp"].mean() if not s.empty else np.nan

    task_rows.append({
        "task_id": int(tid),
        "task_duration_s": dur_s,

        "pupil_mean_bc": pupil_mean,
        "pupil_slope": pupil_slope,
        "blink_time_frac": blink_time_frac,
        "blink_per_min": blink_per_min,

        "fix_rate_hz": fix_rate_hz,
        "fix_dur_mean_s": fix_dur_mean_s,
        "sac_rate_hz": sac_rate_hz,
        "sac_amp_mean_px": sac_amp_mean_px
    })

task_features = pd.DataFrame(task_rows).sort_values("task_id").reset_index(drop=True)
display(task_features)


,task_id,task_duration_s,pupil_mean_bc,pupil_slope,blink_time_frac,blink_per_min,fix_rate_hz,fix_dur_mean_s,sac_rate_hz,sac_amp_mean_px
0,1,358.990243,0.522891,0.000691,0.086539,14.039379,1.791135,0.471891,1.454079,219.622147
1,2,209.060412,1.165456,0.000205,0.059017,13.201926,1.894189,0.458705,1.511525,197.351515
2,3,389.439403,0.588936,-0.000222,0.067974,12.941680,2.154379,0.394670,1.720422,199.446432
3,4,464.259411,0.620132,-0.000132,0.116114,9.563619,1.432389,0.574591,1.038213,160.520821
4,5,456.654522,0.863760,0.001050,0.023232,6.438127,1.151855,0.443323,0.915353,165.621567
5,6,407.495757,0.695545,-0.000255,0.033748,13.104431,2.508002,0.345782,2.007383,230.752001
6,7,381.418928,0.570387,-0.000119,0.108719,15.888042,2.231143,0.357596,1.782817,253.831161
7,8,264.633045,0.385534,-0.000282,0.018133,7.935517,1.496412,0.620149,1.239452,176.636879


In [ ]:
task_features["participant_id"] = "P01"


In [ ]:
task_features = task_features.merge(
    perf[["task_id", "Accuracy"]],
    on="task_id",
    how="left"
)


In [ ]:
task_features = task_features[[
    "participant_id",
    "task_id",
    "task_duration_s",

    "pupil_mean_bc",
    "pupil_slope",
    "blink_time_frac",
    "blink_per_min",

    "fix_rate_hz",
    "fix_dur_mean_s",
    "sac_rate_hz",
    "sac_amp_mean_px",

    "Accuracy"
]]
display(task_features)


,participant_id,task_id,task_duration_s,pupil_mean_bc,pupil_slope,blink_time_frac,blink_per_min,fix_rate_hz,fix_dur_mean_s,sac_rate_hz,sac_amp_mean_px,Accuracy
0,P01,1,358.990243,0.522891,0.000691,0.086539,14.039379,1.791135,0.471891,1.454079,219.622147,0.50
1,P01,2,209.060412,1.165456,0.000205,0.059017,13.201926,1.894189,0.458705,1.511525,197.351515,1.00
2,P01,3,389.439403,0.588936,-0.000222,0.067974,12.941680,2.154379,0.394670,1.720422,199.446432,0.33
3,P01,4,464.259411,0.620132,-0.000132,0.116114,9.563619,1.432389,0.574591,1.038213,160.520821,0.00
4,P01,5,456.654522,0.863760,0.001050,0.023232,6.438127,1.151855,0.443323,0.915353,165.621567,0.29
5,P01,6,407.495757,0.695545,-0.000255,0.033748,13.104431,2.508002,0.345782,2.007383,230.752001,0.29
6,P01,7,381.418928,0.570387,-0.000119,0.108719,15.888042,2.231143,0.357596,1.782817,253.831161,0.33
7,P01,8,264.633045,0.385534,-0.000282,0.018133,7.935517,1.496412,0.620149,1.239452,176.636879,NaN
